# Web scraping: FDA warning letters

In this notebook, we're going to write some code to scrape [tables of data on FDA warning letters issued in 2018](https://www.fda.gov/ICECI/EnforcementActions/WarningLetters/2018/default.htm).

We'll also talk about how we might extend this idea to get warning letters from other years, as well.

First, let's think through the process and decide what tools we'll need. Our goal is to fetch a web page (`requests`), parse the HTML (`bs4`) and write to a local file (`csv`). Let's import what we need:

In [ ]:
# csv

# requests

# BeautifulSoup from bs4


### Now we noodle

The data table of warning letters is spread across multiple pages. What happens when you click "Next" or a page number? The URL changes from this:

[`https://www.fda.gov/ICECI/EnforcementActions/WarningLetters/2018/default.htm`](https://www.fda.gov/ICECI/EnforcementActions/WarningLetters/2018/default.htm)

to this:

[`https://www.fda.gov/ICECI/EnforcementActions/WarningLetters/2018/default.htm?Page=2`](https://www.fda.gov/ICECI/EnforcementActions/WarningLetters/2018/default.htm?Page=2)

What's happening is, a [_parameter_](https://en.wikipedia.org/wiki/Query_string) is being appended to the URL. This gives the database instructions about how to formulate a response to your browser's request: Show me the letters on page 2.

What happens when you specify `Page=1`?

[`https://www.fda.gov/ICECI/EnforcementActions/WarningLetters/2018/default.htm?Page=1`](https://www.fda.gov/ICECI/EnforcementActions/WarningLetters/2018/default.htm?Page=1)

We're back to the first page. This is good news, because it means we can iterate over the number of pages in the results, starting with Page 1, and grab what we need.

What happens when we specify a page number that doesn't exist?

[`https://www.fda.gov/ICECI/EnforcementActions/WarningLetters/2018/default.htm?Page=100`](https://www.fda.gov/ICECI/EnforcementActions/WarningLetters/2018/default.htm?Page=100)

We get a page that includes the text "No Current Postings Available" -- something we'll remember as we write our scraper.

### Save files locally

It's good practice to save the HTML files you want to scrape. So we're going to do that here -- save a copy of each page of results on the FDA site.

First, let's establish a few variables:
- The base URL we'll start from
- The naming pattern of the local files we're going to save -- we're going to use the `format` method of [string formatting](../reference/String%20formatting.ipynb)

In [ ]:
BASE_URL = 'https://www.fda.gov/ICECI/EnforcementActions/WarningLetters/2018/default.htm'
LOCAL_FILE_PATT = 'fda-warning-letters-{pagenum}.html'
CSV_FILE = 'fda-warning-letters.csv'

There are a bunch of different strategies we could use to grab these pages. Here's what we'll try today:
1. Get a list of page numbers that you could navigate to, and grab the largest one
2. Loop over a range of numbers from 1 to the largest number in the pagination list (which we just grabbed)
3. Retrieve each page of results
4. If that page exists -- in other words, if the web page doesn't say "No Current Postings Available" -- save it; if not, break out of the loop because we're done (this is just an extra sanity check)

Why, you ask, don't we just loop over the numbers 1-5? We know that there are only 5 pages. Great question! 

Generally, you want to avoid hard-coding numbers in situations where numbers could change. As the FDA adds more warning letters, the database will return more than 5 pages of results, and our script would miss those.

So first, let's grab the page and isolate the element with the maximum page number. Then we'll know how many pages we need to loop over.

In a new tab, crack open the page to view the source code. Then use `requests` to get the page and `bs4` to parse the HTML.

In [ ]:
# use requests to grab the web page

# turn it into soup


Find the list of page numbers at the bottom -- I Ctrl-F'd for "Previous" to find it. Turns out it's an unordered list (`ul`) with the class `pagination-clean`.

We can use BeautifulSoup's `find()` method to target that element. We'll supply the name of the element and a dictionary of attributes.

In [ ]:
# find the pagination list

# print it


We want to isolate the numbers and grab the biggest. Couple ways to skin this cat, but today, let's pull the text of each item in that `ul` into a Python list -- only if it's a number, though! -- and find the biggest one.

Three new-to-us functions are going to help us out here:
- [`isnumeric()`](https://www.tutorialspoint.com/python/string_isnumeric.htm), a string method that checks whether the contents of a string are numeric ('4' => True, 'Hello!' => False)
- [`int()`](https://docs.python.org/3/library/functions.html#int), a function to coerce a value to an integer
- [`max()`](https://docs.python.org/3/library/functions.html#max), a function to get the biggest number out of a list

We're going to use a _for loop_ to iterate over the items (`li`) inside the list (`ul`).

👉 For a refresher on for loops, [see this notebook](../reference/Python%20data%20types%20and%20basic%20syntax.ipynb#for-loops).

In [ ]:
# an empty list to hold the page numbers


# loop over the items in the pagination list

    
    # get the text inside the tag and strip whitespace

    
    # if it's numeric

        
        # coerce to an integer (`int()`) and `append()` to our list


# create a new variable with the biggest number in that list


'''
~ BONUS CONTENT ~

A one-liner to do this would use something called a "list comprehension"
https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions

max_page_num = max([int(x.string.strip()) for x in pagination.find_all('li')
                    if x.string.strip().isnumeric()])

'''

# print the max page number


So now we have the number of pages we need to loop over. We'll use the [`range()`](https://docs.python.org/3/library/functions.html#func-range) function to loop over the range of numbers we're interested in -- keeping in mind that the second number you give to the range function is the _first_ number that you _don't_ want to include. In other words, we need to add 1 to our `max_page_num` variable to get what we need.

We want to hit pages 1-5, including 5.

To demonstrate this concept:

In [ ]:
for i in range(1, max_page_num):
    print(i)

In [ ]:
for i in range(1, max_page_num+1):
    print(i)

Five requests to a server isn't really a big deal, but just to be courteous we'll pause for a second between requests. For that, we'll use `sleep()`, a method in Python's built-in `time` module. Let's import `time` now.

In [ ]:
# import time


Now, to save our pages. We'll loop over a list of page numbers (1 to max page number) and request the FDA page of results associated with that page number and save to file.

Along the way, if we see the 'no current postings available' text, something went wrong and we want to break out of the loop. To do this, we'll use a [`break`](https://docs.python.org/3/reference/simple_stmts.html#break).

👉 For more information on _if_ statements, [check out this notebook](../reference/Python%20data%20types%20and%20basic%20syntax.ipynb#if-statements).

In [ ]:
# loop over the numbers 1 to max_page_num+1

    
    # get the page, specifying the 'Page' parameter

    
    # `break` out of the loop if the "nothing here" text shows up



    # otherwise, save the page

        # create the filename using the pattern we specified above

        # open that file in write mode

            # and write the text to the file


    # print something to let us know it's working

    
    # pause 1 second


### Scrape the HTML

Now that we've saved the HTML locally, let's get to work scraping it. Let's open the first one as a test and turn it into soup.

In [ ]:
# open the first saved HTML page in read mode

    # read in the contents to a variable

    # turn the HTML into soup


Next, look for the data we want to scrape. It's in a `<table>` element with the ID `WarningLetter_sortid`. (It also appears to be the only table on the page.)

We can use the `find()` method, again, to target it.

In [ ]:
# find the table by id

# print the table


We'll start by looping over the rows (`<tr>`) in the table and seeing what we can pull out. Remember: The `find_all()` method returns a list that we can iterate over.

In [ ]:
# loop over the rows in the table

    # and print each one


Perf. Let's use list slicing to skip the header row, though, and start extracting the data.

In [ ]:
# loop over the table rows, skipping the first [0] one

    
    # get a list of table data elements in this row

    
    # first one [0] has the date
    # get the string attribute and strip whitespace

    
    # second one [1] has the company

    
    # URL also in second one; prepend the base URL

    
    # third one [2] has the office


    # fourth one [3] has the subject

    
    # fifth one [4] has the closeout date
    # using `text` instead of `string` because the text is actually inside the nested `a`

    
    # print it to see what we've got

    # print a divider to make it easier to scan


Cool! Let's try it with the other files. How might we get a list of the HTML files we downloaded earlier? If you didn't know already, how might you formulate your search terms for Google? (I might start with something like "[python get specific files in directory](https://www.google.com/search?q=python+get+specific+files+in+directory)," which leads me to [a StackOverflow question](https://stackoverflow.com/questions/3964681/find-all-files-in-a-directory-with-extension-txt-in-python) from someone who was trying to find all of the `.txt` files in a directory.)

Appears that [`glob`](https://docs.python.org/3/library/glob.html) is our horse here. Let's import `glob` and target our files:

In [ ]:
# import glob

# use glob wildcard to get a list of results pages

# print to make sure we've got what we need


Winner winner, chicken dinner. Now we can loop over each one and extract the data we need from each.

In [ ]:
# loop over the results pages

    
    # open the page, read the HTML, turn it into soup




    # find the table


    # loop over the table rows, skipping the first [0] one


        # get a list of table data elements in this row


        # first one [0] has the date


        # second one [1] has the company


        # URL also in second one; prepend the base URL


        # third one [2] has the office


        # fourth one [3] has the subject


        # fifth one [4] has the closeout date
        # using `text` instead of `string` because the text is actually inside the nested `a`


        # print it to see what we've got

        # print a divider for clarity


Neato. What data are we extracting? The fields on the table are: `Letter Issue`, `Date`, `Company Name`, `Issuing Office`, `Subject`, `Close Out Date`. The company name also has a link to the actual letter, so we'd want to grab that, too. Let's define the field names for our CSV in a list:

In [ ]:
headers = ['company', 'issue_date', 'letter_url', 'issuing_office', 'subject', 'closeout_date']

Now instead of just printing the rows, we want to write them out to a CSV.

Let's put it all together:

In [ ]:
# open a new csv file to write to
# specify write mode, newline=''

    
    # create a writer object, specifying the fieldnames

    
    # write the header row

    
    # loop over the HTML pages

        # open the page, read the HTML, turn it into soup



        
        # find the table


        # loop over the table rows, skipping the first [0] one


            # get a list of table data elements in this row


            # first one [0] has the date


            # second one [1] has the company


            # URL also in second one; prepend the base URL


            # third one [2] has the office


            # fourth one [3] has the subject


            # fifth one [4] has the closeout date
            # using `text` instead of `string` because the text is actually inside the nested `a`


            # write row to csv


So there we have it. At this point, I would go back and refactor the code to break up tasks into discrete [functions](../reference/Functions.ipynb).

### Next up: How to get multiple years' worth of data?

A class discussion + extra credit assignment.